In [15]:
import audible_plot as ap
import ccxt
import numpy as np
import pandas as pd
import pandas_ta as ta

binanceusdm = ccxt.binanceusdm()

In [25]:
candles = np.array(binanceusdm.fetch_ohlcv("BTCUSDT", "4h"))

In [34]:
data = pd.DataFrame(
    data=candles[:, 1:],
    index=pd.to_datetime(pd.Series(candles[:, 0], name="timestamp"), unit="ms"),
    columns=["open", "high", "low", "close", "volume"],
)
data = pd.concat(
    [
        data,
        ta.bbands(data["close"], 20),
        ta.aroon(data["high"], data["low"], 14),
        ta.macd(data["close"]),
        pd.Series(data["high"] - data["low"], name="candle_w"),
        pd.Series(data["close"] - data["open"], name="candle_result"),
    ],
    axis=1,
)
data = data.dropna()
data

,open,high,low,close,volume,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,AROOND_14,AROONU_14,AROONOSC_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,candle_w,candle_result
timestamp,,,,,,,,,,,,,,,,,,
2024-08-17 08:00:00,59221.9,59450.1,59026.0,59106.5,13687.324,56851.173761,58850.760,60850.346239,6.795447,0.563948,35.714286,28.571429,-7.142857,-130.449094,143.528720,-273.977814,424.1,-115.4
2024-08-17 12:00:00,59106.5,59638.8,59065.6,59357.6,18213.242,56986.290026,58774.280,60562.269974,6.084260,0.663122,28.571429,21.428571,-7.142857,-86.732839,149.795980,-236.528819,573.2,251.1
2024-08-17 16:00:00,59357.7,59690.2,59265.3,59446.3,12678.586,57263.048485,58683.560,60104.071515,4.841259,0.768474,21.428571,14.285714,-7.142857,-44.418020,153.688640,-198.106659,424.9,88.6
2024-08-17 20:00:00,59446.2,59474.4,59122.1,59468.5,7323.194,57259.114416,58687.140,60115.165584,4.866571,0.773581,14.285714,7.142857,-7.142857,-8.988255,151.294723,-160.282979,352.3,22.3
2024-08-18 00:00:00,59468.5,60249.8,59258.5,59308.5,34857.959,57257.680244,58709.865,60162.049756,4.946987,0.706115,7.142857,100.000000,92.857143,6.109005,133.113587,-127.004582,991.3,-160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-02 08:00:00,69813.0,69846.2,69413.0,69602.5,9919.876,68237.284018,70722.375,73207.465982,7.027736,0.274681,71.428571,14.285714,-57.142857,-183.171549,-236.993139,53.821589,433.2,-210.5
2024-11-02 12:00:00,69602.6,69700.0,69022.8,69394.0,25701.195,68133.661042,70579.835,73026.008958,6.931651,0.257614,64.285714,7.142857,-57.142857,-214.200972,-214.418049,0.217077,677.2,-208.6
2024-11-02 16:00:00,69393.9,69610.9,69221.6,69502.6,11944.529,68056.633105,70455.215,72853.796895,6.808813,0.301421,57.142857,0.000000,-57.142857,-227.407488,-182.099652,-45.307836,389.3,108.7


In [35]:
chart = ap.AudibleChart(
    data=data,
    config=[
        ap.SeriesConfig(
            key="close",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(ap.ToneGenerator.WaveType.sawtooth),
                volume=0.1,
            ),
        ),
        ap.SeriesConfig(
            key="volume",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(),
                volume=0.5,
                pan=0.3,
            ),
            is_extra=True,
        ),
        ap.SeriesConfig(
            is_extra=True,
            key="aroon_zero",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(
                    wave_type=ap.ToneGenerator.WaveType.triangle
                ),
                pan=0.8,
            ),
            range=ap.FixedRange(-100, 100),
        ),
        ap.SeriesConfig(
            is_extra=True,
            key="AROONOSC_14",
            renderer=ap.ConditionalRenderer(
                condition=lambda v: v > 60 or v <= -60,
                mapper=lambda v: 1 if v > 50 else -1,
                renderer=ap.PitchDataRenderer(
                    enable_transitions=True,
                    generator=ap.ToneGenerator(
                        wave_type=ap.ToneGenerator.WaveType.triangle
                    ),
                    pan=0.3,
                ),
            ),
            range=ap.FixedRange(-1, 1),
        ),
        ap.SeriesConfig(
            is_extra=True,
            key="rsi",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(
                    wave_type=ap.ToneGenerator.WaveType.triangle
                ),
                pan=0.3,
            ),
            range=ap.FixedRange(-0, 100),
        ),
        ap.SeriesConfig(
            is_extra=True,
            key="MACDs_12_26_9",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(
                    wave_type=ap.ToneGenerator.WaveType.triangle
                ),
                pan=-0.8,
            ),
        ),
        ap.SeriesConfig(
            is_extra=True,
            key="MACD_12_26_9",
            renderer=ap.PitchDataRenderer(
                enable_transitions=True,
                generator=ap.ToneGenerator(
                    wave_type=ap.ToneGenerator.WaveType.triangle
                ),
                pan=0.8,
            ),
        ),
    ],
    frequency_range=ap.FixedRange(300, 800),
    sample_rate=44100,
)

window = chart.window(slice(-100, None))
window.play()

In [36]:
data.to_clipboard()